# Loading results from multinest

In [ ]:
# Setup packages
%run load_results_multinest.py

In [ ]:
df.head(5)

In [ ]:
finished = df[['mw', 'config_sigma', 'config_detector', 'n_fit_parameters', 'config_nlive', 'config_save_intermediate', 'config_earth_shielding']].copy()
finished.to_csv('../finished_sims.csv', index=False)

In [ ]:
delete_empty(no_result, False)


In [ ]:
mask = df['config_nlive'] < 500
df[mask]

In [ ]:
delete_with_mask(df, mask)

In [ ]:
all_notes = np.unique(df.config_notes.values)
for note in all_notes:
    print(f'{np.sum(df["config_notes"] == note)}\t{note}')

In [ ]:
mask = df['config_notes'] == 'migdal_04_038'
df_copy = df[~mask].copy()
df_copy

In [ ]:
dfs = []
for mass in np.unique(df_copy['mw']):
    mask1 = df_copy['mw'] == mass
    for sigma in np.unique(df_copy[mask1]['config_sigma']):
        mask2 = df_copy[mask1]['config_sigma'] == sigma
        sel3 = df_copy[mask1][mask2]['config_detector']
        for det in np.unique(sel3):
            mask3 = sel3 == det
            sel4 = df_copy[mask1][mask2][mask3]['n_fit_parameters']
            print(mass, sigma, det, '\n\n\n')
            for n_ in np.unique(sel4):
                mask4 = sel4 == n_
                sel5 = df_copy[mask1][mask2][mask3][mask3][mask4]['config_earth_shielding']
                for shield in np.unique(sel5):                    
                    mask5 = sel5 == shield
                    sel6 = df_copy[mask1][mask2][mask3][mask3][mask4][mask5]['config_save_intermediate']
                    for intm in np.unique(sel6):
                        mask6 = sel6 == intm
                        this_df = df_copy[mask1][mask2][mask3][mask3][mask4][mask5][mask6]
#                         print(mass, sigma, det, n_, shield, intm, len(this_df))
                        if len(this_df) > 1:
                            del_mask = this_df_copy['config_start'] != np.max(this_df_copy['config_start'])
                            delete_with_mask(this_df, del_mask, False)
                            dfs.append(this_df)
                        for k in this_df.sort_values('config_start')['item'].values:
                            overlay_hist_confidence_info(k)

In [ ]:
sel_items = [70,223]
for k in sel_items:
    overlay_hist_confidence_info(k)

In [ ]:
def get_binrange(i):
    return results[i]['config']['prior']['log_mass']['range'],
                     results[i]['config']['prior']['log_cross_section']['range']
    
def combine_posteriors(items):
    assert len(items) == 2, 'only combine two posteriors'
    i = 
    bin_range = []
    two_confidence_plot(sel_items, bin_range = bin_range, text_box=True, nsigma=2, nbins=50)
    ax = plt.gca()
    plt.text(1.6, 1, 'ing', transform=ax.transAxes, fontsize=12, bbox=dict(facecolor="white", boxstyle="round"), verticalalignment='top')
    if bin_range:
        plt.xlim(*bin_range[0])
        plt.ylim(*bin_range[1])

In [ ]:
bin_range

In [ ]:
def two_confidence_plot(items, text_box=False, bin_range=None, nsigma=2, nbins=50):
    fig, ax = plt.subplots(figsize=(8, 6))
    if bin_range == None:
        bin_range = [results[items[0]]['config']['prior']['log_mass']['range'],
                     results[items[0]]['config']['prior']['log_cross_section']['range']
                     ]
    hists = {}
    for k, item in enumerate(items):  # , 78, 110
        x, y = get_p_i(item)
        # Make a 2d normed histogram
        hists[k], xedges, yedges = np.histogram2d(x, y, bins=nbins, range=bin_range, normed=True)
    H = hists[0] * hists[1]
    if True:
        norm = H.sum()  # Find the norm of the sum
        # Set contour levels
        contour3 = 0.99
        contour2 = 0.95
        contour1 = 0.68

        # Take histogram bin membership as proportional to Likelihood
        # This is true when data comes from a Markovian process
        def objective(limit, target):
            w = np.where(H > limit)
            count = H[w]
            return count.sum() - target

        target1 = norm * contour1
        level1 = scipy.optimize.bisect(objective, H.min(), H.max(), args=(target1,))
        levels = [level1]
        if nsigma > 1:
            target2 = norm * contour2
            level2 = scipy.optimize.bisect(objective, H.min(), H.max(), args=(target2,))
            levels.append(level2)
            if nsigma > 2:
                target3 = norm * contour3
                level3 = scipy.optimize.bisect(objective, H.min(), H.max(), args=(target3,))
                levels.append(level3)
            if nsigma > 3:
                print('Nsigma too big')
        levels.reverse()
        levels.append(H.max())

        # Find levels by summing histogram to objective

        # Pass levels to normed kde plot
        def av_levels(x):
            return [(x[i] + x[i + 1]) / 2 for i in range(len(x) - 1)]

        if levels[0] == levels[1]:
            print("ERRRRRRRRR\n\n")
            print(levels)
            levels[0] /= 1.01
            levels = np.unique(levels)
            print(levels)
        sns_ax = sns.kdeplot(x, y, shade=True, ax=ax, n_levels=levels, cmap="viridis", normed=True,
                             cbar=False, vmin=levels[0], vmax=levels[-1])
        kwargs = {}
        if k == 0:
            kwargs['label'] = 'best fit'
        plt.scatter(np.mean(x), np.mean(y), c='black',
                    marker='+', **kwargs)
        if k == 0:
            kwargs['label'] = 'benchmark value'
        plt.scatter(results[item]['config']['mw'],
                    results[item]['config']['sigma'], c='blue',
                    marker='x',
                    **kwargs)
        if k == 0:
            cbar = ax.figure.colorbar(sns_ax.collections[0])
            cbar.set_ticks(av_levels(np.linspace(0, 1, nsigma + 1)))
            col_labels = ['$3\sigma$', '$2\sigma$', '$1\sigma$'][3 - nsigma:]
            cbar.set_ticklabels(col_labels)
            cbar.set_label("Posterior probability")
    
    secax = ax.secondary_xaxis('top', functions=(pow10, np.log10))
    
    if 'migd' in results[items[0]]['config']['detector']:
        x_ticks = [0.01, 0.1, 1, 3, 5]
    else:
        x_ticks = [15, 25, 50, 100, 250, 500, 1000]
    for x_tick in x_ticks:
        ax.axvline(np.log10(x_tick), alpha=0.1)
    secax.set_ticks(x_ticks)
    plt.xlim(np.log10(x_ticks[0]), np.log10(x_ticks[-1]))
    plt.xlabel("$\log_{10}(M_{\chi}$ $[GeV/c^{2}]$)")
    secax.set_xlabel("$M_{\chi}$ $[GeV/c^{2}]$")
    plt.ylabel("$\log_{10}(\sigma_{S.I.}$ $[cm^{2}]$)")
    plt.legend(loc='upper right')

    if text_box:
        plt.text(0.05, 0.95, text_box, transform=ax.transAxes, alpha=0.5,
                 bbox=dict(facecolor="white", boxstyle="round"))

In [ ]:
del_mask = (df.config_detector == 'Xe_migd')
df[del_mask]
delete_with_mask(df, del_mask, False)

In [ ]:
for k in tqdm(np.flip(df_copy.sort_values('config_start')['item'].values)):
    overlay_hist_confidence_info(k)
    

In [ ]:
for k in df_copy['item'].values:
    overlay_hist_confidence_info(k)
    
# assert False

In [ ]:
earth_shielding = df[(df['config_earth_shielding'] == True) & (df['config_detector'] == 'Xe_migd_bg')]
for i in earth_shielding['item'].values:
    overlay_hist_confidence_info(i)

In [ ]:
0


In [ ]:
i = 0
while True:
    i+=1
    if i<10:
        continue
    print(i)
    if i > 100:
        break
    